In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops

In [2]:
import opendatasets as od
import pandas as pd
od.download("https://www.kaggle.com/datasets/koryakinp/fingers")

Skipping, found downloaded files in ".\fingers" (use force=True to force download)


In [3]:
import os
import cv2
import numpy as np

dataset_dir = 'C:/Users/yugan/Downloads/archive/fingers/test'
images = []
labels = []
for filename in os.listdir(dataset_dir):
    img = cv2.imread(os.path.join(dataset_dir, filename), cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (64, 64))
    images.append(img_resized)
    label =int(filename[37])
    labels.append(label)
images=np.array(images)
labels=np.array(labels)
labels

array([5, 5, 3, ..., 4, 3, 3])

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(images,labels,test_size=0.15,random_state=42)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [5]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train, num_classes=6)
Y_test = to_categorical(y_test, num_classes=6)

In [6]:
print ("Y_train shape: " + str(Y_train.shape))
print ("Y_test shape: " + str(Y_test.shape))

Y_train shape: (3060, 6)
Y_test shape: (540, 6)


In [7]:
def convolutional_model(input_shape):
    input_img = tf.keras.Input(shape=input_shape)
    model = tf.keras.Sequential([
        tfl.Conv2D(8, (4, 4), strides=(1, 1), padding='same', input_shape=input_shape),
        tfl.Activation('relu'),
        tfl.MaxPooling2D((8, 8), strides=(8, 8), padding='same'),
        tfl.Conv2D(16, (2, 2), strides=(1, 1), padding='same'),
        tfl.Activation('relu'),
        tfl.MaxPooling2D((4, 4), strides=(4, 4), padding='same'),
        tfl.Flatten(),
        tfl.Dense(6, activation='softmax')
    ])
    return model

In [8]:
conv_model = convolutional_model((64, 64, 1))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
conv_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 8)         136       
                                                                 
 activation (Activation)     (None, 64, 64, 8)         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 8)          0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          528       
                                                                 
 activation_1 (Activation)   (None, 8, 8, 16)          0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 16)         0         
 2D)                                                    

In [9]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train[:1800], Y_train[:1800]))
train_dataset = train_dataset.batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test[:300], Y_test[:300]))
test_dataset = test_dataset.batch(batch_size)

In [12]:
history = conv_model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
57/57 [==============================] - 1s 26ms/step - loss: 0.2849 - accuracy: 0.9028 - val_loss: 0.4611 - val_accuracy: 0.8567
Epoch 2/10
57/57 [==============================] - 1s 25ms/step - loss: 0.2714 - accuracy: 0.9122 - val_loss: 0.4494 - val_accuracy: 0.8633
Epoch 3/10
57/57 [==============================] - 3s 60ms/step - loss: 0.2609 - accuracy: 0.9172 - val_loss: 0.4311 - val_accuracy: 0.8700
Epoch 4/10
57/57 [==============================] - 4s 65ms/step - loss: 0.2497 - accuracy: 0.9239 - val_loss: 0.4133 - val_accuracy: 0.8767
Epoch 5/10
57/57 [==============================] - 4s 65ms/step - loss: 0.2410 - accuracy: 0.9244 - val_loss: 0.4006 - val_accuracy: 0.8933
Epoch 6/10
57/57 [==============================] - 4s 66ms/step - loss: 0.2298 - accuracy: 0.9272 - val_loss: 0.3801 - val_accuracy: 0.9067
Epoch 7/10
57/57 [==============================] - 4s 70ms/step - loss: 0.2199 - accuracy: 0.9306 - val_loss: 0.3682 - val_accuracy: 0.9133
Epoch 8/10
57

In [13]:
import random
import cv2
from matplotlib import pyplot as plt

numbers = list(range(1, 540))
random_array = random.sample(numbers, 10)

fig = plt.figure(figsize=(10, 7))

for i, idx in enumerate(random_array):
    img_resized = cv2.resize(x_test[idx], (64, 64))
    y_pred = np.argmax(conv_model.predict(np.expand_dims(x_test[idx], axis=0),verbose=0))
    ax = fig.add_subplot(1, 10, i+1)
    ax.imshow(img_resized, cmap='gray')
    ax.axis('off')
    ax.set_title("pred = " + str(y_pred))

plt.tight_layout()
plt.show()